In [1]:
import numpy as np

In [2]:
with open('./data/movie-xids.npy', 'rb') as f:
    Xids = np.load(f, allow_pickle=True)
with open('./data/movie-xmask.npy', 'rb') as f:
    Xmask = np.load(f, allow_pickle=True)
with open('./data/movie-labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle=True)

In [3]:
Xids.shape, Xmask.shape, labels.shape

((156060, 512), (156060, 512), (156060, 5))

In [4]:
labels

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [5]:
import tensorflow as tf

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [7]:
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [8]:
# {input_ids, attention_mask}, outputs
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

In [9]:
dataset = dataset.map(map_func)

In [10]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [11]:
batch_size = 16

In [12]:
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

In [13]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [14]:
split = 0.9

In [15]:
size = int((Xids.shape[0] / batch_size) * split)

In [16]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [18]:
tf.data.Dataset.save(train_ds, './data/train_ds')
tf.data.Dataset.save(val_ds, './data/val_ds')

In [19]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))